In [ ]:
import os
import sys

# Obtenir le répertoire de travail courant
current_dir = os.getcwd()

# Obtenir le répertoire parent de `maps` (qui est `src`)
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Ajouter `src` au chemin de recherche des modules
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [ ]:
import pandas as pd
import numpy as np
import folium.features

from useful_methods.data_processing import extract_data
from useful_methods.neighbours_delaunay.graphs import delaunay_graph
from useful_methods.neighbours_delaunay.miscellaneous_for_neighbouring import mean_distance_to_NN
from useful_methods.ihm.maps.mapUtils import *
from city_utils import *

In [ ]:
df = pd.read_csv("../../database/data.csv", sep=";", decimal=',')

In [ ]:
df_extracted = extract_data(df, provider='Orange', techno='4g', min_info=False)
G, pos = delaunay_graph(df_extracted)

In [ ]:
df_countryside = pd.read_csv("../../database/data_countryside.csv", sep=";", decimal=',')

In [ ]:
df_pop = pd.read_csv("../../database/data_population.csv", sep=";", decimal=',')

In [ ]:
# for com in df_extracted['nom_com'].unique():
#     df_extracted.loc[df_extracted['nom_com']==com, 'countryside'] = (1 if (df_countryside.loc[df_countryside['COM_NOM']==com, 'UU_ID']=='SO').all() else 0)

In [ ]:
for com in df_extracted['nom_com'].unique():
    df_extracted.loc[df_extracted['nom_com']==com, 'countryside'] = (1 if (df_pop.loc[df_pop['Commune']==com, 'PMUN']<7000).all() else 0)

In [ ]:
df_extracted.head()

In [ ]:
map = folium.Map(location=np.mean(df_extracted[['latitude','longitude']], axis=0), zoom_start=7, tiles="Cartodb Positron")
points = folium.FeatureGroup(f"Points ({len(G)})").add_to(map)

for bs_id, latitude, longitude in df_extracted[['latitude', 'longitude']].itertuples():
    color = 'gray' if (df_extracted['countryside'][bs_id]==1) else 'magenta'
    points.add_child(folium.CircleMarker(location=[latitude, longitude], color=color, radius=1, popup=f"{bs_id}"))

folium.LayerControl().add_to(map)

map.save("../../out/maps/countryside_v2.html")

Comparison with other methods

In [ ]:
def mean_distance_to_city_level(v):
    if (v>4):
        return 4
    if (v>2):
        return 3
    if (v>1):
        return 2
    return 1

In [ ]:
labels = df_extracted['countryside'].apply(lambda v : -1 if(v==1) else 0)

mean_distances = mean_distance_to_NN(df_extracted[['x', 'y']], n_neighbours=3)
labels_3NN_detailed = mean_distances.apply(mean_distance_to_city_level)
labels_3NN = labels_3NN_detailed.apply(lambda v : 1 if(v<=2) else -1)

In [ ]:
labels_dbs = city_detection(df_extracted[['x', 'y']])['labels']
labels_hdb = city_detection_enhanced(df_extracted[['x', 'y']])['labels']

In [ ]:
def labelsToColor(l1, l2): 
    if ((l1==-1) and (l2==-1)):
        return 'green'
    if((l1==-1) or (l2==-1)):
        return 'blue'
    return 'red'

In [ ]:
colors = pd.Series(data=[labelsToColor(l, l3NN) for l, l3NN in zip(labels, labels_3NN)], index=df_extracted.index)
map1 = folium.Map(location=np.mean(df_extracted[['latitude','longitude']], axis=0), zoom_start=7, tiles="Cartodb Positron")

for ind, latitude, longitude in df_extracted[['latitude', 'longitude']].itertuples():
    com = df_extracted['nom_com'][ind]
    popup = f"{com} [{df_pop.loc[df_pop['Commune']==com,'PMUN'].values[0]} hab] (label, 3-NN) :\n({labels[ind]!=-1},{labels_3NN[ind]!=-1})"
    map1.add_child(folium.RegularPolygonMarker(location=[latitude, longitude], fill_opacity=0.2, radius=1.5, color = colors[ind], popup=popup))

map1.save("../../out/maps/city-detection-comparison_3NN_v2.html")

In [ ]:
colors = pd.Series(data=[labelsToColor(l, ldbs) for l, ldbs in zip(labels, labels_dbs)], index=df_extracted.index)
map2 = folium.Map(location=np.mean(df_extracted[['latitude','longitude']], axis=0), zoom_start=7, tiles="Cartodb Positron")

for ind, latitude, longitude in df_extracted[['latitude', 'longitude']].itertuples():
    popup = f'ville (label, DBScan) :\n({labels[ind]!=-1},{labels_dbs[ind]!=-1})'
    map2.add_child(folium.RegularPolygonMarker(location=[latitude, longitude], fill_opacity=0.2, radius=1.5, color = colors[ind], popup=popup))

map2.save("../../out/maps/city-detection-comparison_DBScan_v2.html")

In [ ]:
colors = pd.Series(data=[labelsToColor(l, lhdb) for l, lhdb in zip(labels, labels_hdb)], index=df_extracted.index)
map3 = folium.Map(location=np.mean(df_extracted[['latitude','longitude']], axis=0), zoom_start=7, tiles="Cartodb Positron")

for ind, latitude, longitude in df_extracted[['latitude', 'longitude']].itertuples():
    popup = f'ville (label, HDBScan) :\n({labels[ind]!=-1},{labels_hdb[ind]!=-1})'
    map3.add_child(folium.RegularPolygonMarker(location=[latitude, longitude], fill_opacity=0.2, radius=1.5, color = colors[ind], popup=popup))

map3.save("../../out/maps/city-detection-comparison_HDBScan_v2.html")

## A new new new database

In [ ]:
df_freq = pd.read_csv("../../database/data_frequency.csv", sep=";", decimal=',')